In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip',sep='\t')
test_df=pd.read_csv('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip',sep='\t')

In [ ]:
train_df.head()

In [ ]:
from nltk.tokenize.treebank import TreebankWordTokenizer

In [ ]:
tbwt=TreebankWordTokenizer()
def tokenizer(row):
    ltokens=tbwt.tokenize(row['Phrase'])
    #print(row['Phrase'])
    tok_sen=' '.join(ltokens)
    #print(ltokens)
    return tok_sen

In [ ]:
train_df['token']=train_df.apply(tokenizer,axis=1)
test_df['token']=test_df.apply(tokenizer,axis=1)

In [ ]:
train_df

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(row):
    #print(row['token'])
    split=row['token'].split(' ')
    lem_sen=' '.join([lemmatizer.lemmatize(w) for w in split])
    #print(lem_sen)
    return lem_sen

In [ ]:
train_df['lemma']=train_df.apply(lemmatize,axis=1)
test_df['lemma']=test_df.apply(lemmatize,axis=1)

In [ ]:
train_df.head()

In [ ]:
from keras.preprocessing.text import Tokenizer
token=Tokenizer()
token.fit_on_texts(train_df['lemma'])

In [ ]:
X_train=token.texts_to_sequences(train_df['lemma'])
X_train

In [ ]:
def mlen(row):
    s=row['lemma'].split(' ')
    return len(s)

train_df['len']=train_df.apply(mlen,axis=1)
train_df.head()

In [ ]:
test_df['len']=test_df.apply(mlen,axis=1)
max(test_df['len'])

In [ ]:
max(train_df['len'])

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_train,maxlen=56)

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#ohe=OneHotEncoder()
Y_train=train_df.iloc[:,3]
#Y_train=Y_train.values
#Y_train=Y_train.reshape(-1,1)
#Y_tr=ohe.fit_transform(Y_train)

In [ ]:
ytr=pd.get_dummies(Y_train.values)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM

In [ ]:
len(token.word_index)

In [ ]:
model=Sequential()
model.add(Embedding(14220,300))
model.add(LSTM(150))
model.add(Dense(5,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
import tensorflow as tf

In [ ]:
xtr=tf.cast(X_train,tf.int32)
ytr=tf.cast(ytr,tf.int32)

In [ ]:
model.fit(xtr,ytr,batch_size=256,epochs=15)

In [ ]:
X_test=token.texts_to_sequences(test_df['lemma'])
X_test=pad_sequences(X_test,maxlen=56,padding='post')

In [ ]:
X_test.shape

In [ ]:
s=model.predict(X_test)
s=np.argmax(s,axis=1)
print(s)
s=pd.DataFrame(s)
s['PhraseId']=test_df['PhraseId']
s.columns=['Sentiment','PhraseId']
s=s[['PhraseId','Sentiment']]
s.to_csv('submissions.csv',index=False)